# Logistic Regression from scratch

Our second model is logistic regression and will be doing a binary classification

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn import datasets 
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Load data
data = datasets.load_breast_cancer()
X = data['data']
y = data['target']

In [ ]:
#np.random.seed(0)
m,n = X.shape
theta = np.random.randn(n) # initialization of theta

#add bias terms to X and theta
X_with_bias = np.c_[np.ones(m),X] 
theta_with_bias = np.insert(theta,0,0)

In [ ]:
#Split Data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_with_bias,y,test_size = 0.25)

In [ ]:
#Scaling X

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
#Define Sigmoid function
def sigmoid(x):
    z = 1/(1 + np.exp(-x)) 
    return z


In [ ]:
##Function for training data
def log_reg(theta,X,y,alpha):
    cost = []
    for i in range(2000):
        h = np.dot(X,theta)
        z = sigmoid(h)    
        J = (-1/m)*np.sum(y*np.log(z)+(1-z)*np.log(1-z))
        cost.append(J)
        error = z - y
        grad = (1/m)*np.sum(error*X.T)
        theta =  theta - grad*alpha
        if i % 500 == 0:
            print(i, J)
    pred = (z>=0.5).astype(int)
    return pred,theta,cost

In [ ]:
#Training the data
pred_train, theta,cost = log_reg(theta_with_bias,X_train_scaled,y_train,0.001)

In [ ]:
#Plotting Cost FUnction to see if it is working properly
plt.plot(cost)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_pred = sigmoid(np.dot(theta,X_test_scaled.T))
test_pred = (test_pred>=0.5).astype(int)

In [ ]:
acc_train = accuracy_score(y_train,pred_train)
acc_train

In [ ]:
acc_test = accuracy_score(y_test,test_pred)
acc_test